<a href="https://colab.research.google.com/github/HeidiHyeji/data_standard/blob/main/first_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai faiss-cpu numpy rouge_score -qqq

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.7 MB/s eta 0:00:00


In [2]:
import openai
openai.api_key = ""

In [3]:
from openai import OpenAI
from rouge_score import rouge_scorer

# ChatGPT API 호출
client = OpenAI(api_key=openai.api_key)

def get_gpt_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content.strip()

# 데이터 표준 적용한 프롬프트
standard_prompt = "표준화된 데이터: [표준 데이터 입력 예시], 이 데이터를 기반으로 XX 결과를 예측해주세요."

# 데이터 표준 미적용 프롬프트
non_standard_prompt = "비표준화된 데이터: [비표준 데이터 입력 예시], 이 데이터를 기반으로 XX 결과를 예측해주세요."

standard_response = get_gpt_response(standard_prompt)
non_standard_response = get_gpt_response(non_standard_prompt)

# 참조 답변
reference_answer = "참조 정답 예시"

# ROUGE로 평가
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
standard_scores = scorer.score(reference_answer, standard_response)
non_standard_scores = scorer.score(reference_answer, non_standard_response)

print(f"표준 데이터셋 ROUGE 점수: {standard_scores}")
print(f"비표준 데이터셋 ROUGE 점수: {non_standard_scores}")


표준 데이터셋 ROUGE 점수: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0, recall=0, fmeasure=0)}
비표준 데이터셋 ROUGE 점수: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0, recall=0, fmeasure=0)}


In [18]:
import openai
import faiss
import numpy as np

# OpenAI API 키 설정"

# 표준 데이터 메타데이터 예시
standard_metadata = [
    "고객_아이디 (Customer ID, 정수형)",
    "구매_날짜 (Purchase Date, YYYY-MM-DD)",
    "상품_코드 (Product Code, 문자열)"
]

# OpenAI 임베딩 함수 정의
def get_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return np.array(response['data'][0]['embedding'])

# 표준 데이터 임베딩
standard_embeddings = np.array([get_embedding(text) for text in standard_metadata])

# FAISS 인덱스 생성 및 데이터 추가
dimension = standard_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(standard_embeddings)

# 사용자 데이터 메타데이터 예시 (비표준)
user_metadata = "고객번호 (CustomerNumber, 숫자형)"

# 사용자 데이터 임베딩
user_embedding = get_embedding(user_metadata).reshape(1, -1)

# 추천 표준 데이터 찾기 (가장 유사한 데이터 1개 추천)
distances, indices = index.search(user_embedding, k=1)
recommended_standard = standard_metadata[indices[0][0]]

print("🎯 추천된 표준 데이터:", recommended_standard)

APIRemovedInV1: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
